In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import cufflinks as cf
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

cf.go_offline()

%matplotlib inline
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

from neuralforecast.auto import AutoTCN
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.core import NeuralForecast
from neuralforecast.models import TCN

from datetime import datetime, timedelta
import plotly.graph_objects as go
import matplotlib.colors as mcolors
from neuralforecast.losses.pytorch import MQLoss
from tqdm import tqdm

In [2]:
# Load and process the MEASLES_ARIZONA data
df = pd.read_csv("outbreaks_disease_location.csv")
value_columns = [str(i) for i in range(60)]
series_values = df[value_columns].fillna(0).astype(float)
start_dates = pd.to_datetime(df["start_date"])

# Shuffle and split
shuffled_indices = df.sample(frac=1, random_state=42).index
split_point = int(0.8 * len(df))
train_indices = shuffled_indices[:split_point]
test_indices = shuffled_indices[split_point:]

In [3]:
train_records = []
for i, row in series_values.iloc[train_indices].iterrows():
    dates = pd.date_range(start="2000-01-01", periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        train_records.append({"unique_id": f"Y{i+1}", "ds": dates[t], "y": value})
df_train = pd.DataFrame(train_records)

test_start_dates = start_dates.loc[test_indices] - pd.Timedelta(weeks=4)
df_test_all = []

for idx in test_indices:
    start_date = test_start_dates.loc[idx]
    row = series_values.loc[idx]
    dates = pd.date_range(start=start_date, periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        df_test_all.append({"unique_id": f"Y_{idx}", "ds": dates[t], "y": value})

df_test_all = pd.DataFrame(df_test_all)

In [4]:
df_train["date"] = pd.to_datetime(df_train["ds"])
df_train.set_index("date", inplace = True)

df_test_all["date"] = pd.to_datetime(df_test_all["ds"])
df_test_all.set_index("date", inplace = True)

In [5]:
df_test_all

,unique_id,ds,y
date,,,
2022-08-13,Y_5607,2022-08-13,20775.0
2022-08-20,Y_5607,2022-08-20,16570.0
2022-08-27,Y_5607,2022-08-27,13995.0
2022-09-03,Y_5607,2022-09-03,12145.0
2022-09-10,Y_5607,2022-09-10,10686.0
...,...,...,...
1939-05-20,Y_7270,1939-05-20,0.0
1939-05-27,Y_7270,1939-05-27,0.0
1939-06-03,Y_7270,1939-06-03,0.0


In [6]:
class FixedModelTCNProcessor:
    def __init__(self, dates=[]):
        self.dates  = dates
        self.forecasts, self.eval_pairs = [], []
        self.testset = None
        self.nf, self.model = None, None
        self.selected_input_size = None
        self.maes, self.mses, self.mapes, self.nmses = [], [], [], []
        self.metrics_df  = pd.DataFrame(columns=["Unique_id","Reference Date","MAE","MSE","MAPE","NMSE"])
        self.display_df  = pd.DataFrame(columns=["Unique_id","Reference Date","Target End Date","GT","Quantile","Prediction"])
        self.wis_df      = pd.DataFrame()

    # ──────────────────────────────────────────
    def create_fixed_model(self, df_train, df_test, h: int, freq: str, level=[]):
        min_len   = df_train.groupby("unique_id").size().min()
        max_input = max(8, min_len - h - 1)
        self.testset = df_test

        def cfg(trial):
            return {
                "input_size"  : trial.suggest_int("input_size", 8, max_input),
                "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
                "batch_size"  : trial.suggest_categorical("batch_size", [16,32,64]),
                "random_seed" : trial.suggest_int("random_seed", 1, 99999),
                "start_padding_enabled": True,
                "accelerator": "gpu", "devices": 2, "strategy": "auto"
            }

        model = AutoTCN(h=h, backend="optuna",
                        loss=MQLoss(level=level) if level else None,
                        config=cfg)

        self.nf = NeuralForecast(models=[model], freq=freq)
        self.nf.fit(df=df_train)
        self.model = self.nf.models[0].model
        self.selected_input_size = self.model.hparams["input_size"]

    # ──────────────────────────────────────────
    def sliding_batch_by_index(self, h: int = 4):
        uid_dates = {u: g["ds"].sort_values().to_numpy()
                     for u, g in self.testset.groupby("unique_id")}
        T, s = min(len(d) for d in uid_dates.values()), self.selected_input_size
        rows = []

        for t in tqdm(range(s, T - h + 1), desc="index t"):
            batch, refs = [], []
            for uid, dates in uid_dates.items():
                win, ref = dates[t-s:t], dates[t-1]
                df_u = self.testset[self.testset["unique_id"] == uid]
                df_in = df_u[df_u["ds"].isin(win)].copy()
                if len(df_in) == s:
                    df_in["unique_id"] = uid
                    batch.append(df_in); refs.append((uid, ref))
            if not batch: continue

            fcst = self.nf.predict(df=pd.concat(batch)).set_index("ds")
            for uid, ref in refs:
                f  = fcst[fcst["unique_id"] == uid].reset_index()
                gt = (self.testset[self.testset["unique_id"] == uid]
                      .set_index("ds").loc[f["ds"]].reset_index())
                merged = pd.merge(f, gt, on=["unique_id","ds"], how="inner")
                merged["Reference Date"] = pd.to_datetime(ref).strftime("%Y-%m-%d")
                rows.append(merged)

        return pd.concat(rows, ignore_index=True)

    # ──────────────────────────────────────────
    def create_display_df(self, merged_df, model_name="AutoTCN"):
        records = []
        for _, g in tqdm(merged_df.groupby("unique_id"), desc="display"):
            uid = g["unique_id"].iloc[0]; ref = g["Reference Date"].iloc[0]
            for col in g.columns:
                if col.startswith(model_name):
                    if col.endswith("median") or col == model_name:
                        q = 0.5
                    elif "-lo-" in col:
                        lvl = int(col.split("-")[-1]); q = (1 - lvl/100)/2
                    elif "-hi-" in col:
                        lvl = int(col.split("-")[-1]); q = 1 - (1 - lvl/100)/2
                    else: continue
                    tmp = g[["ds", col, "y"]].rename(columns={"ds":"Target End Date",
                                                               col:"Prediction",
                                                               "y":"GT"})
                    tmp["Unique_id"] = uid
                    tmp["Reference Date"] = ref
                    tmp["Quantile"] = q
                    records.append(tmp)
        self.display_df = (pd.concat(records)
                           .loc[:,["Unique_id","Reference Date","Target End Date",
                                   "GT","Quantile","Prediction"]]
                           .sort_values(["Unique_id","Reference Date",
                                         "Target End Date","Quantile"])
                           .reset_index(drop=True))

    # ──────────────────────────────────────────
    def efficient_compute_wis(self):
        if self.display_df.empty:
            raise ValueError("Run create_display_df first.")
        recs, df = [], self.display_df.sort_values(
            ["Unique_id","Reference Date","Target End Date","Quantile"])
        for (uid, ref, tgt), grp in tqdm(df.groupby(
                ["Unique_id","Reference Date","Target End Date"]), desc="WIS"):
            gt = grp["GT"].iloc[0]
            preds = grp.set_index("Quantile")["Prediction"]
            if 0.5 not in preds.index: continue
            wis = abs(preds[0.5] - gt)  # AE-only shortcut
            recs.append({"Unique_id":uid,"Reference Date":ref,
                         "Target End Date":tgt,"GT":gt,"WIS":wis})
        self.wis_df = pd.DataFrame(recs)

In [7]:
test_ids = [f"Y_{i}" for i in test_indices]

In [8]:
processor = FixedModelTCNProcessor()

In [9]:
processor.create_fixed_model(df_train, df_test_all, h=4, freq="W-SAT", level = [10,20,30,40,50,60,70,80, 85,90,95])

[I 2025-06-25 16:39:10,377] A new study created in memory with name: no-name-f7e4e7b7-f708-487c-8b87-3d8f9d812af8
[rank: 0] Seed set to 87093
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
LOCAL_RANK: 1 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name            | Type                       | Params | Mode 
-----------------------------------------------------------------------
0 | loss            | MQLoss                     | 23     | train
1 | padder_train    | ConstantPad1d              

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1000` reached.


In [10]:
results = []

In [11]:
processor.selected_input_size

15

In [12]:
all_results = processor.sliding_batch_by_index(h=4)

index t:   0%|          | 0/42 [00:00<?, ?it/s]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t:   2%|▏         | 1/42 [00:34<23:51, 34.91s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t:   5%|▍         | 2/42 [01:08<22:50, 34.26s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


...

index t:  95%|█████████▌| 40/42 [22:59<01:07, 33.72s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t:  98%|█████████▊| 41/42 [23:32<00:33, 33.60s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t: 100%|██████████| 42/42 [24:06<00:00, 34.44s/it]


In [13]:
all_results

,ds,unique_id,AutoTCN-median,AutoTCN-lo-95,AutoTCN-lo-90,AutoTCN-lo-85,AutoTCN-lo-80,AutoTCN-lo-70,AutoTCN-lo-60,AutoTCN-lo-50,...,AutoTCN-hi-40,AutoTCN-hi-50,AutoTCN-hi-60,AutoTCN-hi-70,AutoTCN-hi-80,AutoTCN-hi-85,AutoTCN-hi-90,AutoTCN-hi-95,y,Reference Date
0,2011-01-01,Y_1,3.132855,2.050130,2.239692,2.045792,2.356884,2.458385,2.499907,2.659526,...,3.236453,2.950097,3.498597,3.276565,3.648149,3.547128,3.729919,3.945046,4.47843,2010-12-25
1,2011-01-08,Y_1,3.242585,1.661372,2.046097,1.792222,2.112587,2.322606,2.531757,2.821932,...,3.537158,3.060127,3.877001,3.297281,3.810079,3.651067,4.110472,4.653080,6.28019,2010-12-25
2,2011-01-15,Y_1,3.015598,1.855345,2.122758,2.043020,2.196977,2.330951,2.460706,2.647216,...,3.217371,2.937867,3.518291,3.269913,3.584047,3.377859,3.730071,4.053163,3.68098,2010-12-25
3,2011-01-22,Y_1,2.863966,0.807982,1.253191,1.606199,2.017574,2.619829,2.771101,2.970995,...,3.388866,2.593119,3.358875,3.182272,3.805043,4.242726,4.371981,4.049060,4.43182,2010-12-25
4,2021-08-28,Y_1000,3917.174072,-677.307373,105.492188,-1051.543213,1550.428711,2245.655518,2704.915771,3685.729004,...,5604.334473,1078.458130,4435.467773,1612.430176,3823.817383,3271.547119,3883.689209,6268.229492,42880.00000,2021-08-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362875,1955-04-16,Y_9998,0.998268,0.016454,0.330380,0.453888,0.624516,0.853077,0.912857,0.861370,...,1.076288,1.017932,1.226165,1.148208,1.367803,1.625250,1.563157,1.476765,0.00000,1955-03-19
362876,1957-03-02,Y_9999,0.768886,-0.399060,-0.105721,0.031349,0.030465,0.072324,0.258141,0.239099,...,0.985291,1.182727,1.246482,1.744485,1.915273,1.836343,2.019193,2.272439,0.00000,1957-02-23
362877,1957-03-09,Y_9999,0.990582,-1.117482,-0.549683,-0.325013,-0.070222,-0.057811,0.119831,0.052965,...,0.818548,0.950828,1.321248,1.359226,1.674070,1.563783,1.886996,2.258651,0.00000,1957-02-23
362878,1957-03-16,Y_9999,0.883797,-0.962874,-0.407665,-0.288945,-0.024785,0.017201,0.201627,0.117458,...,0.835669,0.988265,1.162722,1.541533,1.844376,1.697860,1.962932,2.234898,0.00000,1957-02-23


In [14]:
all_results.to_csv('TCN_concatenated.csv')

In [15]:
df = all_results.rename(columns={
    "ds"        : "Target End Date",
    "unique_id" : "Unique_id",
    "y"         : "GT"
})

# long‐format all quantile columns (everything that starts with AutoNBEATS-)
quantile_cols = [c for c in df.columns
                 if c.startswith("AutoTCN-")]

df_long = df.melt(
    id_vars = ["Unique_id", "Reference Date", "Target End Date", "GT"],
    value_vars = quantile_cols,
    var_name = "QuantileLabel",
    value_name = "Prediction"
)

def to_quantile(label: str) -> float:
    if "median" in label: return 0.5
    if "-lo-" in label:
        lvl = int(label.split("-")[-1]); return (1 - lvl/100) / 2
    if "-hi-" in label:
        lvl = int(label.split("-")[-1]); return 1 - (1 - lvl/100) / 2
    raise ValueError(label)

df_long["Quantile"] = df_long["QuantileLabel"].apply(to_quantile)

rows = []

group_iter = df_long.groupby(["Unique_id", "Reference Date", "Target End Date"])
for (uid, ref_date, tgt_date), g in tqdm(group_iter,
                                         total=len(group_iter),
                                         desc="point-wise metrics"):

    gt = g["GT"].iloc[0]
    preds = g.set_index("Quantile")["Prediction"]

    if 0.5 not in preds:          # need the median
        continue

    # point errors
    y_pred = preds[0.5]
    mae  = abs(y_pred - gt)
    mse  = (y_pred - gt) ** 2
    mape = abs((y_pred - gt) / gt) if gt else np.nan
    nmse = np.nan                # single point ⇒ var = 0

    # WIS for this timestamp
    qs  = sorted(q for q in preds.index if q != 0.5)
    n   = len(qs) // 2
    ae  = mae
    int_scores = 0
    for i in range(n):
        lo_q, hi_q = qs[i], qs[-(i + 1)]
        lo, hi     = preds[lo_q], preds[hi_q]
        alpha      = hi_q - lo_q
        int_scores += (hi - lo) \
                    + (2/alpha) * max(lo - gt, 0) \
                    + (2/alpha) * max(gt - hi, 0)

    wis = (ae + int_scores) / (1 + n)

    rows.append({
        "Unique_id"      : uid,
        "Reference Date" : ref_date,
        "Target End Date": tgt_date,
        "GT"             : gt,
        "Prediction"     : y_pred,
        "MAE"            : mae,
        "MSE"            : mse,
        "MAPE"           : mape,
        "NMSE"           : nmse,
        "WIS"            : wis
    })

pointwise_df = pd.DataFrame(rows).sort_values(
    ["Unique_id", "Reference Date", "Target End Date"])

point-wise metrics: 100%|██████████| 362880/362880 [03:14<00:00, 1866.69it/s]


In [16]:
pointwise_df

,Unique_id,Reference Date,Target End Date,GT,Prediction,MAE,MSE,MAPE,NMSE,WIS
0,Y_1,2010-12-25,2011-01-01,4.47843,3.132855,1.345575,1.810571,0.300457,NaN,7.324986
1,Y_1,2010-12-25,2011-01-08,6.28019,3.242585,3.037605,9.227043,0.483680,NaN,15.725841
2,Y_1,2010-12-25,2011-01-15,3.68098,3.015598,0.665382,0.442734,0.180762,NaN,3.419771
3,Y_1,2010-12-25,2011-01-22,4.43182,2.863966,1.567854,2.458165,0.353772,NaN,7.245649
4,Y_1,2011-01-01,2011-01-08,6.28019,4.474652,1.805538,3.259966,0.287497,NaN,12.921752
...,...,...,...,...,...,...,...,...,...,...
362875,Y_9999,1957-02-16,1957-03-16,0.00000,0.602381,0.602381,0.362862,NaN,NaN,4.958175
362876,Y_9999,1957-02-23,1957-03-02,0.00000,0.768886,0.768886,0.591185,NaN,NaN,3.512562
362877,Y_9999,1957-02-23,1957-03-09,0.00000,0.990582,0.990582,0.981253,NaN,NaN,3.929624
362878,Y_9999,1957-02-23,1957-03-16,0.00000,0.883797,0.883797,0.781097,NaN,NaN,3.669347


In [17]:
pointwise_df.to_csv('TCN_ALL_STATS.csv')